# Interactive Map of Spain

This notebook contains the code to create an interactive `html` map of spain using the `folium` package.

The map was created for site selection in the [Humanotop ALDEA](https://humanotop.earth/aldea/) project. Custom layers with relevant information for the selection (e.g. potential abandonadas, PV potential, ...) are imported from `geojson` files. 
These files were created in advance using the package `geopandas`. 

A description of the map with instruction manual and the sources of the data can be found in the file `map_information.pptx`. 

In [ ]:
# this is to make jupyter interface wider for the map
from IPython.display import display, HTML
display(HTML('<style>.container { width:80% !important; }</style>'))

In [ ]:
import os 
import datetime
import json

import numpy as np
import pandas as pd
import folium
from folium import raster_layers
from folium import plugins

In [ ]:
# path to geojson files containing data for layers
data_path = 'data'

In [ ]:
# load 5x5 km grid for weather data 
with open(os.path.join(data_path, 'raster_5km.geojson')) as f:
        weather_geo = json.load(f)
        
# load 10x10km grid for precipitation data
with open(os.path.join(data_path, 'raster_10km.geojson')) as f:
        prec_geo = json.load(f)

# load spanish communities
with open(os.path.join(data_path, 'communities.geojson')) as f:
        data = json.load(f)
    
# load potential abandondas = abandoned villages
with open(os.path.join(data_path, 'abandonadas.geojson')) as f:
        abandonada = json.load(f)

# load soil classification        
with open(os.path.join(data_path, 'soil.geojson')) as f:
        soil = json.load(f)
        
# load seismic hazard layer
with open(os.path.join(data_path, 'seismic_hazard.geojson')) as f:
        seismic_hazard = json.load(f)

In [ ]:
# read tabular data - weather data
# since the map is loaded for the whole of Spain, the grid/weather data was scaled/averaged to 5x5 km 
# however, the data is also available in a finer resolution of 1x1 km
weather_data = pd.read_csv(os.path.join(data_path, 'weather_data_5km.csv'))
# precipitation data is only available in 10x10 km
prec = pd.read_csv(os.path.join(data_path, 'precipitation_10km.csv'))

In [ ]:
weather_data.head()

In [ ]:
# create a map using the Map() function and the coordinates for Madrid as center
m = folium.Map(location=[40.416775, -3.703790], control_scale=True, zoom_start=13)

#folium.Marker(location, popup=str(geolocator.reverse(str(location[0])+","+str(location[1])))).add_to(m)

#m.add_child(folium.LatLngPopup())

# this shows geocoords of mouse position in upper right corner
fmtr = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"
plugins.MousePosition(position='topright', separator=' | ', prefix="Position:",
              lat_formatter=fmtr, lng_formatter=fmtr).add_to(m)

# tile layer is the underlying map: openstreetmap, terrain, and satellite are added
tile1 = folium.TileLayer('Stamen Terrain', name='Stamen Terrain')
tile1.add_to(m)

tile2 = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Satellite',
        overlay = False,
        control = True
        )
tile2.add_to(m)

# in the following lines, the layers are added
# autonomous communities
style = {'color': 'black', 'fillOpacity': 0}    
communities = folium.GeoJson(data, style_function = lambda x: style, name="Autonomous Communities", popup=folium.GeoJsonPopup(fields=['Community']))
communities.add_to(m)

# potential abandonadas
abandonadas = folium.GeoJson(abandonada, name="Potential Abandonadas", show=False, popup=folium.GeoJsonPopup(fields=['Name', 'Status as of', 'Area [Sq km]', 'Area [Acres]'], localize=False))
abandonadas.add_to(m)


# pv potential with legend
choropleth1 = folium.Choropleth(
    geo_data=weather_geo,
    name='PV Potential',
    data=weather_data,
    columns=['CellCode', 'pv_potential'],
    key_on='feature.properties.CellCode',
    fill_color='YlOrRd',
    nan_fill_color='transparent',
    fill_opacity=0.7,
    line_opacity=0,
    legend_name='PV Potential [kWh/kWp]',
    highlight=False,
    line_color='black',
    show=False
    )

choropleth1.add_to(m)

# wind speed with legend
choropleth2 = folium.Choropleth(
    geo_data=weather_geo,
    name='Wind Speed @10m',
    data=weather_data,
    columns=['CellCode', 'wind_speed'],
    key_on='feature.properties.CellCode',
    fill_color='Blues',
    nan_fill_color='transparent',
    fill_opacity=0.7,
    line_opacity=0,
    legend_name='Wind Speed [m/s]',
    highlight=False,
    line_color='black',
    show=False
    )

choropleth2.add_to(m)

# air temperature with legend
choropleth3 = folium.Choropleth(
    geo_data=weather_geo,
    name='Air Temperature',
    data=weather_data,
    columns=['CellCode', 'temp'],
    key_on='feature.properties.CellCode',
    fill_color='RdBu_r',
    nan_fill_color='transparent',
    fill_opacity=0.7,
    line_opacity=0,
    legend_name='Air Temperature [°C]',
    highlight=False,
    line_color='black',
    show=False
    )
choropleth3.add_to(m)

# precipitation with legend
choropleth4 = folium.Choropleth(
    geo_data=prec_geo,
    name='Mean Annual Precipitation',
    data=prec,
    columns=['CellCode', 'precipitation'],
    key_on='feature.properties.CellCode',
    fill_color='YlGnBu',
    nan_fill_color='transparent',
    fill_opacity=0.7,
    line_opacity=0,
    legend_name='Mean Annual Precipitation [mm]',
    highlight=False,
    line_color='black',
    show=False
    )
choropleth4.add_to(m)

# soil classification
soils = folium.GeoJson(soil, name="FAO90 Soil Classification", show=False, style_function=lambda feature: {
        'fillColor': feature['properties']['color'],
        'color' : feature['properties']['color'],
        'weight' : 1,
        'fillOpacity' : 0.7,
        }, popup=folium.GeoJsonPopup(fields=['FAO90 Type']))
soils.add_to(m)

# seismic hazard
seismic = folium.GeoJson(seismic_hazard, name="Seismic Hazard", show=False, style_function=lambda feature: {
        'fillColor': feature['properties']['color'],
        'color' : 'transparent',
        'weight' : 1,
        'fillOpacity' : 0.7,
        })
seismic.add_to(m)

# below some additional functionalities are added
# search bar
m.add_child(plugins.Geocoder(collapsed=True))
folium.LayerControl().add_to(m)

# area measurement
m.add_child(plugins.MeasureControl())

In [ ]:
# save the generated map as html file
m.save('spain_map.html')